In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy import constants as const
import numpy as np
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

umPerPx = 6.949
dumPerPx = 4E-2
dRelUmPerPx = dumPerPx/umPerPx

def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge

def paramDictatTime(param_dict, time_fs):
    return param_dict['A1']*np.exp(-time_fs/param_dict['tau1']) + param_dict['A2']*np.exp(-time_fs/param_dict['tau2'])
#import data from excel (terrible choice I know, but I want the visualisation and comparability)

def ErrorCorrectionConvoluted(signal, correctionFactor, peakRadiance, dRelUmPerPx, dRelPower):
    '''correctionfactor is already divided with peak radiance\\
    dRelPower = dPower/Power'''
    dPeakRadiance = dRelPower*peakRadiance + 2*peakRadiance*dRelUmPerPx
    return abs(signal*correctionFactor/peakRadiance*dPeakRadiance)

def ErrorCorrectionSimple(signal, correctionFactor, peakRadiance, dRelPower):
    dPeakRadiance = dRelPower*peakRadiance
    return abs(signal*correctionFactor/peakRadiance*dPeakRadiance)

def fitPeaks(x_data, y_data, N_gaussians, centers = [], sigma = [], amplitudes = [], positionBounds = [], ampBounds = [], function = None):
    from scipy.optimize import curve_fit
    startParameters = np.ones(3*N_gaussians)
    min_bounds = []
    max_bounds = []
    if len(centers) != N_gaussians:
        centers = np.linspace(min(x_data), max(x_data),N_gaussians)
    
    if len(amplitudes) != N_gaussians:
        amplitudes = max(np.abs(y_data))/N_gaussians*np.ones(N_gaussians)
    if len(sigma) != N_gaussians:
        sigma = np.ones(N_gaussians)*0.01

    for i in range(N_gaussians):
        startParameters[i*3+1] = sigma[i]
        startParameters[i*3] = centers[i]
        startParameters[i*3+2] = amplitudes[i]

    for i in range(N_gaussians):
        if len(positionBounds) == 0:
            min_bounds.append(min(x_data))
            max_bounds.append(max(x_data))
        else:
            min_bounds.append(positionBounds[i][0])
            max_bounds.append(positionBounds[i][1])
        min_bounds.append(0)
        max_bounds.append(np.inf)
        if len(ampBounds) == 0:
            min_bounds.append(-np.inf)
            max_bounds.append(np.inf)
        else:
            min_bounds.append(ampBounds[0])
            max_bounds.append(ampBounds[1])
    par_bounds = [tuple(min_bounds), tuple(max_bounds)]


    popt, pcov = curve_fit(function, x_data, y_data, 
            p0 = startParameters, 
            bounds = par_bounds,
            method="trf")
    return popt, pcov

def multiGaussianAdditive(x_data, *parameters):
    r'''parameters are parameters[number of gaussian, center/sig/amplitude]'''
    y_out = np.zeros(np.shape(x_data))
    for gaussInd in range(0,len(parameters),3):
        y_out += parameters[gaussInd+2]*np.exp(-np.power(x_data-parameters[gaussInd+0], 2)/(2*np.power(parameters[gaussInd+1],2)))
    return y_out

def multiLorentzianAdditive(x_data, *parameters):
    r'''parameters are parameters[number of gaussian, center/gamma/amplitude]'''
    y_out = np.zeros(np.shape(x_data))
    for cauchyInd in range(0,len(parameters),3):
        #y_out += 1/(np.pi*parameters[cauchyInd+1]*(1+np.power((x_data-parameters[cauchyInd])/parameters[cauchyInd+1],2)))
        #removing normalisation constant, want it to be 1 in center position
        y_out += parameters[cauchyInd+2]/((1+np.power((x_data-parameters[cauchyInd])/parameters[cauchyInd+1],2)))
    return y_out


excel_path = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\CorrectedPump653ProbeWavelengthScan_UV-extended.xlsx"
SHG_wav_timescan = pd.read_excel(excel_path, sheet_name = "2024WavelengthScanParameters", skiprows=[0,1])
SHG_main = pd.read_excel(excel_path, sheet_name='2024WavelengthScan', skiprows=[0,1])
#print(SHG_main)
#print(SHG_main.keys())


wavelengths = SHG_main['Probe wavelength / nm']
#simple correction without map
dOD_SHG_main = SHG_main['cor_dOD / (mOD*m^2/W)']
#correction with map
dOD_SHG_mapCorr = SHG_main['hypothetical correction']
dOD_map_reference = SHG_main['mapReference']
dPumpPower = SHG_main['dRelPumpPower']
peakRad = SHG_main['Pump power density / (W/m^2)']

#map reference correction
dOD_map_reference = abs(dOD_map_reference/np.mean(dOD_map_reference))

dOD_map_reference[7] = np.nan

scan_wavs = SHG_wav_timescan['Probe wavelength / nm']
scan_corrFactors = SHG_wav_timescan['Correction Factor / (W/m^2)^-1']
scan_params = {
    'A1' : SHG_wav_timescan['A1'],
    'A2' : SHG_wav_timescan['A2'],
    'tau1' : SHG_wav_timescan['tau1'],
    'tau2' : SHG_wav_timescan['tau2'],
}

times = np.array([1e2, 1e4, 4e4])



#print(ErrorCorrectionConvoluted(dOD_SHG_main, scan_corrFactors, peakRad, dRelUmPerPx, dPumpPower))
fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1,2), dpi = 288)
colors = ['red', 'green', 'orange', 'royalblue']
#plot 1e2 fs time errorbar
#signalFit = [2.82, 0.14, 1e-3, 2.48, 0.09, 1.3e-3]

Lorentz = True
totalSignalTesting = dOD_map_reference*paramDictatTime(scan_params, 1e2)*scan_corrFactors
bool_mask = np.invert(np.isnan(totalSignalTesting))
totalSignalTesting = np.array(totalSignalTesting[bool_mask])
wavelengths = wavelengths[bool_mask]
scan_params['A1'] = scan_params["A1"][bool_mask]
scan_params['A2'] = scan_params["A2"][bool_mask]
scan_params['tau1'] = scan_params["tau1"][bool_mask]
scan_params['tau2'] = scan_params["tau2"][bool_mask]
dOD_map_reference = dOD_map_reference[bool_mask]
scan_corrFactors = scan_corrFactors[bool_mask]
peakRad = peakRad[bool_mask]
dPumpPower = dPumpPower[bool_mask]
scan_wavs = scan_wavs[bool_mask]
#signalFit, signalCov = fitPeaks(wavToEnergy(wavelengths[1:-2]), totalSignalTesting[1:-2],2, [2.82, 2.48], [0.1, 0.05],[1e-3, 1e-3])

if Lorentz == True:
    signalFit, signalCov = fitPeaks(wavToEnergy(wavelengths[1:-2]), totalSignalTesting[1:-2],3, [2.82, 2.32, 3.25], [0.1,0.03, 0.1], [4e-4, 4e-4, 1e-4], positionBounds=[[2.76, 2.83], [2.3, 2.7], [3.00,3.60]],ampBounds=[0,2e-3], function=multiLorentzianAdditive)
    #signalFit, signalCov = fitPeaks(wavToEnergy(wavelengths[1:-2]), totalSignalTesting[1:-2],2, [2.82, 2.48], [0.1,0.05], [1e-3, 1e-3], function=multiLorentzianAdditive)
else:
    signalFit, signalCov = fitPeaks(wavToEnergy(wavelengths[1:-2]), totalSignalTesting[1:-2],3, [2.82, 2.48, 3.55], [0.1,0.05, 0.02], [1e-3, 1e-3, 2e-4], positionBounds=[[2.76, 2.83], [2.3, 2.7], [3.20,3.60]], function=multiGaussianAdditive)
#plot gaussians
print(signalFit)
print(np.sqrt(np.diag(signalCov)))
wav_range = np.linspace(350,550,500)
if Lorentz == True:
    ax.plot(wav_range, multiLorentzianAdditive(wavToEnergy(wav_range), *signalFit), label=r"$\sum$ fits", color="m", alpha=0.5,linestyle="-")
    for i in range(0,len(signalFit),3):
        ax.plot(wav_range, multiLorentzianAdditive(wavToEnergy(wav_range), *signalFit[i:i+3]), label="fit "+str(int(i/3)), alpha=0.5,linestyle="-")
else:
    ax.plot(wav_range, multiGaussianAdditive(wavToEnergy(wav_range), *signalFit), label="sum gauss")
    for i in range(0,len(signalFit),3):
        ax.plot(wav_range, multiGaussianAdditive(wavToEnergy(wav_range), *signalFit[i:i+3]), label="gauss fit "+str(int(i/3)), linestyle="dashed")


ax.errorbar(wavelengths, totalSignalTesting, yerr = ErrorCorrectionSimple(dOD_map_reference*paramDictatTime(scan_params, 1e2), scan_corrFactors, peakRad, dPumpPower), ls = "None", capsize =2, ecolor=colors[0])

'''
ax.plot(wavelengths, dOD_SHG_main, 'ro', label='standard correction', ls = "None")
ax.plot(wavelengths, dOD_SHG_mapCorr, 'bx', label='map corrected', ls = "None")
#ax.errorbar(wavelengths, dOD_SHG_main, yerr = ErrorCorrectionConvoluted(dOD_SHG_main, scan_corrFactors, peakRad, dRelUmPerPx, dPumpPower), ls = "None", capsize =2)
'''
for ind, time in enumerate(times):
    ax.plot(scan_wavs, dOD_map_reference*paramDictatTime(scan_params, time)*scan_corrFactors, '1', label="%.1f ps" %(time*1e-3), color = colors[ind])
''''''

#plot 310 and 330 nm
ax.plot([310, 330], [0,0], '1', color = colors[-1])

ax.set_xlabel('probe wavelength / nm')
ax.set_ylabel(r'$\mathrm{\Delta A / a.u.}$')

ax.set_xticks(np.arange(300, 701, 50))
secax = ax.secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
secax.set_xticks(np.round(wavToEnergy(np.array([440, 500, 653])), 2))
secax.set_xlabel(r"$\mathrm{E_{probe} / eV}$")
ax.legend()
ax.grid()


plt.show()

### Fitted Zheng on top of our measurement, only one time delay

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy.constants as const
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

#copied from above
def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge


def paramDictatTime(param_dict, time_fs):
    return param_dict['A1']*np.exp(-time_fs/param_dict['tau1']) + param_dict['A2']*np.exp(-time_fs/param_dict['tau2'])
#import data from excel (terrible choice I know, but I want the visualisation and comparability)

def ErrorCorrectionConvoluted(signal, correctionFactor, peakRadiance, dRelUmPerPx, dRelPower):
    '''correctionfactor is already divided with peak radiance\\
    dRelPower = dPower/Power'''
    dPeakRadiance = dRelPower*peakRadiance + 2*peakRadiance*dRelUmPerPx
    return abs(signal*correctionFactor/peakRadiance*dPeakRadiance)

def fitPeaks(x_data, y_data, N_gaussians, centers = [], sigma = [], amplitudes = [], positionBounds = [], ampBounds = [], function = None):
    from scipy.optimize import curve_fit
    startParameters = np.ones(3*N_gaussians)
    min_bounds = []
    max_bounds = []
    if len(centers) != N_gaussians:
        centers = np.linspace(min(x_data), max(x_data),N_gaussians)
    
    if len(amplitudes) != N_gaussians:
        amplitudes = max(np.abs(y_data))/N_gaussians*np.ones(N_gaussians)
    if len(sigma) != N_gaussians:
        sigma = np.ones(N_gaussians)*0.01

    for i in range(N_gaussians):
        startParameters[i*3+1] = sigma[i]
        startParameters[i*3] = centers[i]
        startParameters[i*3+2] = amplitudes[i]

    for i in range(N_gaussians):
        if len(positionBounds) == 0:
            min_bounds.append(min(x_data))
            max_bounds.append(max(x_data))
        else:
            min_bounds.append(positionBounds[i][0])
            max_bounds.append(positionBounds[i][1])
        min_bounds.append(0)
        max_bounds.append(np.inf)
        if len(ampBounds) == 0:
            min_bounds.append(-np.inf)
            max_bounds.append(np.inf)
        else:
            min_bounds.append(ampBounds[0])
            max_bounds.append(ampBounds[1])
    par_bounds = [tuple(min_bounds), tuple(max_bounds)]


    popt, pcov = curve_fit(function, x_data, y_data, 
            p0 = startParameters, 
            bounds = par_bounds,
            method="trf")
    return popt, pcov

def multiLorentzianAdditive(x_data, *parameters):
    r'''parameters are parameters[number of gaussian, center/gamma/amplitude]'''
    y_out = np.zeros(np.shape(x_data))
    for cauchyInd in range(0,len(parameters),3):
        #y_out += 1/(np.pi*parameters[cauchyInd+1]*(1+np.power((x_data-parameters[cauchyInd])/parameters[cauchyInd+1],2)))
        #removing normalisation constant, want it to be 1 in center position
        y_out += parameters[cauchyInd+2]/((1+np.power((x_data-parameters[cauchyInd])/parameters[cauchyInd+1],2)))
    return y_out

def multiGaussianAdditive(x_data, *parameters):
    r'''parameters are parameters[number of gaussian, center/sig/amplitude]'''
    y_out = np.zeros(np.shape(x_data))
    for gaussInd in range(0,len(parameters),3):
        y_out += parameters[gaussInd+2]*np.exp(-np.power(x_data-parameters[gaussInd+0], 2)/(2*np.power(parameters[gaussInd+1],2)))
    return y_out

excel_path = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\CorrectedPump653ProbeWavelengthScan_UV-extended.xlsx"
SHG_wav_timescan = pd.read_excel(excel_path, sheet_name = "2024WavelengthScanParameters", skiprows=[0,1])
SHG_main = pd.read_excel(excel_path, sheet_name='2024WavelengthScan', skiprows=[0,1])
#print(SHG_main)
#print(SHG_main.keys())


wavelengths = SHG_main['Probe wavelength / nm']
#simple correction without map
dOD_SHG_main = SHG_main['cor_dOD / (mOD*m^2/W)']
#correction with map
dOD_SHG_mapCorr = SHG_main['hypothetical correction']
dOD_map_reference = abs(SHG_main['mapReference'])
dPumpPower = SHG_main['dRelPumpPower']
peakRad = SHG_main['Pump power density / (W/m^2)']


scan_wavs = SHG_wav_timescan['Probe wavelength / nm']
scan_corrFactors = SHG_wav_timescan['Correction Factor / (W/m^2)^-1']
scan_params = {
    'A1' : SHG_wav_timescan['A1'],
    'A2' : SHG_wav_timescan['A2'],
    'tau1' : SHG_wav_timescan['tau1'],
    'tau2' : SHG_wav_timescan['tau2'],
}

scan_corrFactors[7] = np.nan
#high res zheng

zheng_high_res_path = r"C:\Users\M\Documents\Books\masterprojectinformation\images\Zheng2020_higherPrecisionRip.csv"
Zheng = pd.read_csv(zheng_high_res_path, sep=",")

ZhengDict = {}
ZhengDict['200fs'] = np.array([Zheng["0.2 ps"][1:], Zheng["Y0.2 ps"][1:]], dtype=float)
ZhengDict['2500fs'] = np.array([Zheng["2.5 ps"][1:], Zheng["Y2.5 ps"][1:]], dtype=float)
ZhengDict['10ps'] = np.array([Zheng["10 ps"][1:], Zheng["Y10 ps"][1:]], dtype=float)
ZhengDict['100ps'] = np.array([Zheng["100 ps"][1:], Zheng["Y100 ps"][1:]], dtype=float)
#print(np.array(ZhengDict["2500fs"][0,:],dtype=int))
#print(np.array(ZhengDict["2500fs"][0,:],dtype=int) == 680)
ZhengModifier = abs(ZhengDict["2500fs"][1,np.array(ZhengDict["2500fs"][0,:],dtype=int) == 679])

fig, axs = plt.subplots(1,1, figsize = plotHelperLatex.figSizer(1,2), dpi=288)
figc, axc = plt.subplots(2,1, figsize = plotHelperLatex.figSizer(1, 1.4), dpi=288)

#times = [2e2, 2.5e3, 1e4, 1e5]
times = [25e3]
colors = ['red', 'green', 'orange', 'royalblue']

indexMeasureModifier = np.array(scan_wavs, int) == 680
modifier_scan_params = {
    'A1' : SHG_wav_timescan['A1'][indexMeasureModifier],
    'A2' : SHG_wav_timescan['A2'][indexMeasureModifier],
    'tau1' : SHG_wav_timescan['tau1'][indexMeasureModifier],
    'tau2' : SHG_wav_timescan['tau2'][indexMeasureModifier],
}
ZhengRefSig = np.array(Zheng["Y2.5 ps"][1:], dtype=float)
ZhengRefWav = np.array(Zheng["2.5 ps"][1:], dtype=float)
bool_mask = np.invert(np.isnan(ZhengRefSig))
ZhengRefSig = np.array(ZhengRefSig[bool_mask])
ZhengRefWav = ZhengRefWav[bool_mask]

measurementModifier = abs(scan_corrFactors*dOD_map_reference[indexMeasureModifier]*paramDictatTime(modifier_scan_params, times[0]))[0]
#print((dOD_map_reference[indexMeasureModifier]*paramDictatTime(scan_params, time))[0])


axc[0].plot(ZhengRefWav, ZhengRefSig/ZhengModifier, linestyle="None", marker = ".", markersize=3 , color="r", label="Zheng 2.5 ps")
#zhengFit, zhengCov = fitPeaks(wavToEnergy(ZhengRefWav), ZhengRefSig/ZhengModifier,2, [2.82, 2.48], [0.2,0.1], [0.5, 0.7], positionBounds=[[2.76, 2.83], [2.3, 2.7]], function=multiLorentzianAdditive)
zhengFit, zhengCov = fitPeaks(wavToEnergy(ZhengRefWav), ZhengRefSig/ZhengModifier,2, [2.82, 2.48], [0.2,0.05], [0.35, 0.7], positionBounds=[[2.76, 2.83], [2.3, 2.7]], function=multiGaussianAdditive)
#axc[1].plot(ZhengRefWav, ZhengRefSig/ZhengModifier, linestyle="None", marker = ".", markersize= 2, color='r')

axs.plot(ZhengRefWav, ZhengRefSig/ZhengModifier, linestyle="None", marker = ".", markersize= 3, color="r")
if True:
    sizeM = 2
    for ind, key in enumerate(ZhengDict):
        break
        if ind < 1:
            continue
        axs.plot(ZhengDict[key][0,:], ZhengDict[key][1,:]/ZhengModifier, linestyle="None", marker = ".", markersize= sizeM, color=colors[ind])
        axc[0].plot(ZhengDict[key][0,:], ZhengDict[key][1,:]/ZhengModifier, linestyle="None", marker = ".", markersize= sizeM, color="r", label="Zheng %s" %(key))
        #zhengFit, zhengCov = fitPeaks(wavToEnergy(ZhengDict[key][0,:]), ZhengDict[key][1,:]/ZhengModifier,2, [2.82, 2.48], [0.2,0.1], [0.5, 0.7], positionBounds=[[2.76, 2.83], [2.3, 2.7]], function=multiLorentzianAdditive)
        zhengFit, zhengCov = fitPeaks(wavToEnergy(ZhengDict[key][0,:]), ZhengDict[key][1,:]/ZhengModifier,2, [2.82, 2.48], [0.2,0.05], [0.35, 0.7], positionBounds=[[2.76, 2.83], [2.3, 2.7]], function=multiGaussianAdditive)
        #axc[1].plot(ZhengDict[key][0,:], ZhengDict[key][1,:]/ZhengModifier, linestyle="None", marker = ".", markersize= sizeM, color=colors[ind])
        break

else:
    sizeM = 1
    for ind, key in enumerate(ZhengDict):
        axs.plot(ZhengDict[key][0,:], ZhengDict[key][1,:]/ZhengModifier, linestyle="-", linewidth = sizeM, marker = "None", color=colors[ind])
        axc[0].plot(ZhengDict[key][0,:], ZhengDict[key][1,:]/ZhengModifier, linestyle="-", linewidth = sizeM,  marker = "None", color=colors[ind])
        axc[1].plot(ZhengDict[key][0,:], ZhengDict[key][1,:]/ZhengModifier, linestyle="-", linewidth = sizeM,  marker = "None", color=colors[ind])
print(zhengFit)
for ind, time in enumerate(times):
    axs.plot(scan_wavs, scan_corrFactors*dOD_map_reference*paramDictatTime(scan_params, time)/measurementModifier, '1', color = "b", label="%.1f ps" %(time*1e-3))
    #axc[0].plot(scan_wavs, scan_corrFactors*dOD_map_reference*paramDictatTime(scan_params, time)/measurementModifier, '1', color = colors[ind], label="%.1f ps" %(time*1e-3))
    axc[1].plot(scan_wavs, scan_corrFactors*dOD_map_reference*paramDictatTime(scan_params, time)/measurementModifier, '1', color = colors[ind], label="%.1f ps" %(time*1e-3))
#plotting 310 and 330 by hand
axs.plot([310, 330], [0,0], '1', color = "b")
#axc[0].plot([310, 330], [0,0], '1', color = colors[-1], label="%.1f ps" %(times[-1]*1e-3))
axc[1].plot([310, 330], [0,0], '1', color = colors[0], label="%.1f ps" %(times[0]*1e-3))

#############plot the fitting of with cauchy functions##############################
y_dataFit = scan_corrFactors*dOD_map_reference*paramDictatTime(scan_params, times[0])/measurementModifier
y_dataFit = y_dataFit[1:-2]
wav_dataFit = scan_wavs[1:-2]
#get rid of nan
bool_mask = np.invert(np.isnan(y_dataFit))
y_dataFit = np.array(y_dataFit[bool_mask])
wav_dataFit = wav_dataFit[bool_mask]

signalFit, signalCov = fitPeaks(wavToEnergy(wav_dataFit), y_dataFit, 3, [2.75, 2.48, 3.15], [0.1,0.04, 0.02], [0.5, 0.5, 0.1], positionBounds=[[2.74, 2.83], [2.3, 2.7], [3.015,3.60]], function=multiLorentzianAdditive, ampBounds=[0,np.inf])


wav_range = np.linspace(350,550,500)
axs.plot(wav_range, multiLorentzianAdditive(wavToEnergy(wav_range), *signalFit), label=r"$\sum$ fits", alpha=0.5,linestyle="-", color = "b")
axc[1].plot(wav_range, multiLorentzianAdditive(wavToEnergy(wav_range), *signalFit), label=r"$\sum$ fits", alpha=0.5,linestyle="-", color = "m")
axc[0].plot(wav_range, multiLorentzianAdditive(wavToEnergy(wav_range), *zhengFit), label=r"$\sum$ fits", alpha=0.5,linestyle="-", color = "m")
#axc[0].plot(wav_range, multiGaussianAdditive(wavToEnergy(wav_range), *zhengFit), label=r"$\sum$ fits", alpha=0.5,linestyle="-", color = "m")
color_fits = ["blue", "green", "purple"]
for i in range(0,len(signalFit),3):
    #break
    #axs.plot(wav_range, multiLorentzianAdditive(wavToEnergy(wav_range), *signalFit[i:i+3]), label="fit "+str(int(i/3)), alpha=0.5,linestyle="-", color = color_fits[int(i/3)])
    axc[1].plot(wav_range, multiLorentzianAdditive(wavToEnergy(wav_range), *signalFit[i:i+3]), label="fit "+str(int(i/3)), alpha=0.5,linestyle="-", color = color_fits[int(i/3)])
    axc[0].plot(wav_range, multiLorentzianAdditive(wavToEnergy(wav_range), *zhengFit[i:i+3]), label="fit "+str(int(i/3)), alpha=0.5,linestyle="-", color = color_fits[int(i/3)])


secax = axs.secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
secax.set_xticks(np.round(wavToEnergy(np.array([440, 500, 653])), 2))
secax.set_xlabel(r"$\mathrm{E_{probe} / eV}$")




axs.set_xlabel('probe wavelength / nm')
axs.set_ylabel(r'relative $\Delta A$ / a.u.')
axs.grid(True)

print(signalFit)

#custom legend
from matplotlib.lines import Line2D
legend_single = [Line2D([0], [0], color = "r", label=" 2.5 ps Zheng et al.", marker=".", linestyle="None"),
                    Line2D([0], [0], color = "b", label="2.5 ps SHG setup", marker="1",linestyle="None"), 
                    Line2D([0], [0], color = "b", label="SHG setup cauchy fit", marker="None",linestyle="-", alpha=0.5)]

axs.legend(handles=legend_single)
legend_itemsZheng = [Line2D([0], [0], color = "r", label=" 2.5 ps Zheng et al.", marker=".", linestyle="None")]
legend_itemsSHG = [Line2D([0], [0], color = "r", label="2.5 ps SHG setup", marker="1",linestyle="None")]
for ind in range(len(times)):
    break
    legend_items.append(Line2D([0], [0], color = colors[ind], label="%.1f ps" %(times[ind]*1e-3), marker="s", linestyle="None"))

#introduce cauchy fits to legend
legend_itemsZheng.append(Line2D([0], [0], label=r"$\sum$ fits", linestyle="-", alpha=0.5, color = "m"))
legend_itemsSHG.append(Line2D([0], [0], label=r"$\sum$ fits", linestyle="-", alpha=0.5, color = "m"))
for l2d in range(3):
    #break
    legend_itemsSHG.append(Line2D([0], [0], label="fit "+str(int(l2d)), linestyle="-", alpha=0.5, color = color_fits[l2d]))
    if l2d < 2:
        legend_itemsZheng.append(Line2D([0], [0], label="fit "+str(int(l2d)), linestyle="-", alpha=0.5, color = color_fits[l2d]))
#axs.legend(handles=legend_items)


for i in range(2):
    secax2 = axc[i].secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
    secax2.set_xticks(np.round(wavToEnergy(np.array([410, 440, 500, 653])), 2))
    secax2.set_xlabel(r"$\mathrm{E_{probe} / eV}$")
    axc[i].set_xlabel('probe wavelength / nm')
    axc[i].set_ylabel(r'relative $\Delta A$ / a.u.')
    #axc[i].legend(handles=legend_items)
    axc[i].grid(visible=True)
axc[1].legend(handles = legend_itemsSHG)
axc[0].legend(handles = legend_itemsZheng)

#axs.set_ylim((0,1.3))
#axs.set_xlim((300, 550))
axc[1].set_xlim((300,550))
axc[1].set_ylim((0,1.3))
axc[0].set_xlim((300,550))
axc[0].set_ylim((0,1.3))

#plt.grid(True)
#plt.show()
plt.tight_layout()

#attempt the alternative moving mean at a width of 10 nm aka +- 5 nm
gaussian = lambda wavs, centerwav: 1/np.sqrt(2*np.pi)/5*np.exp(-(centerwav-wavs)**2/(2*5**2))
